<h1>Renaming Feature Directories</h1>
The goal of this code is to be able to rename musicmap directories from their human-readable labels to their genre codes. This uses the mapping in the musicmap_genres.csv file

In [33]:
import os
import pandas as pd
import json
import re
import string
from datetime import datetime

# === CONFIGURATION ===
csv_path = 'musicmap_genres.csv'                  # Path to the CSV file
base_dir = 'musicmap_processed_output/mfcc_rgb'           # Directory containing the folders
dry_run = False                                     # Set to False to actually rename
log_file = 'rename_log.txt'                        # Log file (human-readable)
reverse_log_file = 'rename_reverse_map.json'       # JSON for reversal

# === LOAD CSV ===
df = pd.read_csv(csv_path)

def normalize_label(text):
    if not isinstance(text, str):
        return ''
    text = text.strip().strip('\u00a0')
    text = text.replace('\u00a0', ' ')
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def match_key(text):
    """
    Strips all whitespace and punctuation for matching.
    """
    if not isinstance(text, str):
        return ''
    text = normalize_label(text).lower()
    text = text.translate(str.maketrans('', '', string.punctuation + ' '))  # Remove punctuation and spaces
    return text

# === BUILD LOOKUP ===
label_to_code = {}
match_key_to_code = {}

for code, label in zip(df['Code'], df['Label']):
    norm_label = normalize_label(label)
    label_to_code[norm_label] = code
    match_key_to_code[match_key(norm_label)] = (norm_label, code)

# === INITIALIZE LOGS ===
log_entries = []
reverse_map = {}  # For reversal

# === PERFORM RENAME OR SIMULATE ===
for dirname in os.listdir(base_dir):
    dirpath = os.path.join(base_dir, dirname)
    if not os.path.isdir(dirpath):
        continue

    mk = match_key(dirname)

    if mk in match_key_to_code:
        label, new_name = match_key_to_code[mk]
        new_path = os.path.join(base_dir, new_name)

        if os.path.exists(new_path):
            msg = f"SKIP: Target '{new_name}' already exists for directory '{dirname}'"
            log_entries.append(msg)
            continue

        msg = f"RENAME: '{dirname}' → '{new_name}'  (Label: '{label}')"
        log_entries.append(msg)
        reverse_map[new_name] = dirname  # For reversing later

        if not dry_run:
            os.rename(dirpath, new_path)
    else:
        msg = f"WARNING: '{dirname}' not found in Label column (match_key: '{mk}')"
        log_entries.append(msg)

# === WRITE LOG FILE ===
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
with open(log_file, 'w') as f:
    f.write(f"# Rename Log - {timestamp}\n\n")
    f.write('\n'.join(log_entries))
print(f"Log written to {log_file}")

# === SAVE REVERSAL MAP ===
if not dry_run:
    with open(reverse_log_file, 'w') as f:
        json.dump(reverse_map, f, indent=2)
    print(f"Reversal map saved to {reverse_log_file}")
else:
    print("Dry-run mode enabled: no directories were actually renamed.")


Log written to rename_log.txt
Reversal map saved to rename_reverse_map.json


And to be able to reverse the naming

In [ ]:
import os
import json

# === CONFIGURATION ===
base_dir = 'path_to_directories'
reverse_log_file = 'rename_reverse_map.json'
dry_run = True  # Set to False to actually rename back

# === LOAD REVERSAL MAP ===
with open(reverse_log_file, 'r') as f:
    reverse_map = json.load(f)

# === RENAME BACK ===
for new_name, original_name in reverse_map.items():
    new_path = os.path.join(base_dir, new_name)
    orig_path = os.path.join(base_dir, original_name)

    if not os.path.exists(new_path):
        print(f"SKIP: '{new_name}' does not exist.")
        continue

    if os.path.exists(orig_path):
        print(f"SKIP: Cannot rename to '{original_name}' — already exists.")
        continue

    print(f"RENAME BACK: '{new_name}' → '{original_name}'")
    if not dry_run:
        os.rename(new_path, orig_path)

if dry_run:
    print("ℹ️ Dry-run mode enabled: no directories were actually renamed.")
